<a href="https://colab.research.google.com/github/erin-baggs/DuckweedMicrobes/blob/main/IndivSamples_16S_Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Google co-lab notebook for 16S rRNA amplicon data processing

Erin Baggs erinbaggs95@berkeley.edu

Adapted for duckweed  from https://github.com/d-j-k/puntseq Lara Urban, Andre Holzer, J Jotautas Baronas, Michael Hall, Philipp Braeuninger-Weimer, Michael J Scherm, Daniel J Kunz, Surangi N Perera, Daniel E Martin-Herranz, Edward T Tipper, Susannah J Salter, and Maximilian R Stammnitz

**What is Google Co-Lab**

Colab is a free Jupyter notebook environment that runs entirely in the cloud. Jupyter notebooks can integrate many programming languages like Python, PHP, R, C#. Allowing you to create and share documents that contain live code, equations, visualizations, and narrative text.

[Background](https://colab.research.google.com/?utm_source=scs-index) on using google colab from google if interested. 



**Text Cell Basics** 
This is a **text cell**. You can **double-click** to edit these cells with your own notes. Text cells
use markdown syntax. To learn more, see our [markdown
guide](/notebooks/markdown_guide.ipynb).You can add either text or code cells using the the top toolbar. 

**Code Cell Basics**

Below is a **code cell**. Once the toolbar button indicates CONNECTED, click in the cell to select it and execute the contents in the following ways:

Click the Play icon in the left gutter of the cell;
Type Cmd/Ctrl+Enter to run the cell in place;
Type Shift+Enter to run the cell and move focus to the next cell (adding one if none exists); or
Type Alt+Enter to run the cell and insert a new code cell immediately below it.
There are additional options for running some or all cells in the Runtime menu.

You can edit the code if you double-click on the cell. You can try this with the cell below by changing the print message. If you want to add a note (not code to run) directly to the code then use `#` to tell colab to ignore the rest of the line. 

To get the most out of the workshop try to edit the code where possible to check your understanding of what is going on. 

If your changes prevent the code from running you will no longer see a tick at the side of the code cell and the output will likely contain a series of warning or error messages. 

Before we start so you can keep any changes you make a copy of this notebook in google drive.  File -> Save a copy in drive.




**Where is our data from?**

For background on the data we are using today check out the bioRxiv [doc]<insert link>

**Getting started with Google Co-Lab**


To start we need to link our use of this notebook to our google drive. To do this we will run the grey code block below by clicking on the block and pressing the play button that appears or by clicking on it and pressing shift + enter. 

It will first ask you trust to agree that you want to run the notebook as its not authored by google but our lab.

This will generate a pop-up that asks you if you want google colab to access drive. Select your UC Berkeley google drive account and then on the next pop-up I normally give it all permissions available and press allow. Once approved your drive should start loading. 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Get data for tutorial**

To access the data needed for this tutorial we need to download [SILVA](https://www.arb-silva.de/) database files (tax_slv_ssu_138.1.txt, SILVA_138.1_SSURef_NR99_tax_silva.fasta, seqid2taxid.map, nodes.dmp, names.dmp).

The demultiplexed fasta reads which allow you to skip to **Mapping to reference database** can be found on ncbi PRJNA785658 (SRR18059370-SRR18059372). 

Versions of barcode demultiplex scripts used in processing raw nanopore reads can be found on [github](https://github.com/krasileva-group/Duckweed-Microbiome.git).  

Once you have downloaded the data put it in your google drive or load into the colab session. 


#Processing raw nanopore reads (skip if have fasta)

In [ ]:
!pip uninstall --yes gdown # After running this line, restart Colab runtime.

In order for the colab session to forget the old version of the software gdown we need to go to Runtime at the top tool bar and press the option 'Restart Runtime' before we continue with the next cell. 

In [ ]:
!pip install gdown -U --no-cache-dir

import gdown

url = '<insert your url to reads>'

gdown.download_folder(url)
!mv /content/16SColab /content/drive/MyDrive

**Setup output directories for output**
In order to have somewhere to  put the reads we first run the `mkdir` command in the cell below. This creates a folder in which we can put our location-based reads. 


Here we are prefixing our commands with an `!` to the tell the workbook to run this using a shell rather than python. 
We are making directories to keep our data organized in our drives. 

In [ ]:
!mkdir /content/drive/MyDrive/16SColab/sam/
!mkdir /content/drive/MyDrive/16SColab/demultiplexed/
!mkdir /content/drive/MyDrive/16SColab/fasta/
!mkdir /content/drive/MyDrive/16SColab/abundanceTables/
!mkdir /content/drive/MyDrive/16SColab/samlog
!mkdir /content/drive/MyDrive/16SColab/samsorted
!mkdir /content/drive/MyDrive/16SColab/samoutput

**Install tools needed for data processing**

Google Colab doesnt suppport conda environments that are often used to load tools needed. Instead we will use pip install together with downloads from github to access needed tools.

In [ ]:
!pip install git+https://github.com/rrwick/Porechop.git  # just so pomoxis will install cleanly
!pip install medaka pomoxis aplanat intervaltree==3.0.2
# install samtools from source
!wget https://github.com/samtools/samtools/releases/download/1.10/samtools-1.10.tar.bz2
!tar -xjf samtools-1.10.tar.bz2
!cd samtools-1.10 && ./configure --prefix=/usr/local/ && make && make install
!wget https://github.com/lh3/minimap2/releases/download/v2.17/minimap2-2.17_x64-linux.tar.bz2
!tar -xjf minimap2-2.17_x64-linux.tar.bz2
!cp /content/minimap2-2.17_x64-linux/minimap2 /usr/local/bin/
!pip install requests 
!pip install zenodo-get


Now we have installed tools I like to double check that they work. One way to do this is to ask for the tool help page which will tell you what options are available. 

Once cells run you can clear the output from your screen so you dont have to scroll through them. You can do this by hovering your cursor over the top of the square brackets and then draw it down until a cross appears.  

In [ ]:
!ls

In [ ]:
!porechop -h 

Now that we know Porechop has installed properly we want to find exactly were it is installed as we need to customize some of the code to convert it from removing Oxford nanopore barcode to the PacBio barcodes we used in our experiment. 

In [ ]:
!pip list -v | grep [Pp]orechop

The location is given in the third column as a list of folders (aka directories) seperated by slashes. If the location printed from cell above is different to the one below then copy and paste the location in the third column followed immediatley by a slash and the name of the tool as printed in the first column. We can test this by using `ls` to list the files inside of the porechop folder. If you get a list containing some .py files (thesen are python scripts) then it worked. 

In [ ]:
!ls /usr/local/lib/python3.7/dist-packages/porechop/

Now we can replace the native barcode file in porechop with the pacbio barcode version that is part of the data you have put in your drive. 


In [ ]:
!cp  /content/drive/MyDrive/16SColab/scripts/adapter-barcode-fwd.py /usr/local/lib/python3.7/dist-packages/porechop/adapters.py

To save us some work we are providing one fastq file which contains all the reads from the two runs on   consecutive days  using a single library prep . 

Porechop is able to find and sort reads by the 5' and 3' barcodes that allow us to pool multiple samples into one sequencing run. As well as barcodes porechop identifies and removes ONT adaptors sequences to prevent them leading to mismapping. 

As our samples have unique fwd and rev barcodes we first have to seperate the fwd barcodes and then we re-run porechop with the rev barcodes to identify the combination of barcodes for each sample. 

As the whole dataset takes >1hr to demultiplex I would sugggest to instead run the script on a subsample of the data (unless you want to run it whilst you are doing something else and not just during class**). You can adjust the number of reads to include in your subsample. As fastq files have 4 lines per read I would suggest to try a multiple of 4. To adjust double click on the code block below and change the number after `-n ` it will be interesting to see affects of sample sizes on the outcomes.



```
# head -n 
```


**Notes for running on full dataset not subsample 
Do not run cell below instead skip to porechop command and swap the input filename from flongle-subsample.fastq to flongle-combined.fastq


In [ ]:
!head -n 181048 /content/drive/MyDrive/16SColab/RawReads/flongle-combined.fastq > /content/drive/MyDrive/16SColab/RawReads/flongle-subsample.fastq

In [ ]:
!porechop -i /content/drive/MyDrive/16SColab/RawReads/flongle-subsample.fastq -b /content/drive/MyDrive/16SColab/flongle-demult1

In [ ]:
!cp  /content/drive/MyDrive/16SColab/scripts/adapter-barcode-rev.py /usr/local/lib/python3.7/dist-packages/porechop/adapters.py

Now we have identified forward barcodes we are now going to search for the reverse ones, to do this we need to give porechop a new barcode list which we do with the copy command 

```
cp
```





Next we need to create somewhere to put the reverse demultiplexed reads. To do this we use 

```
mkdir
```
to make a new folder/directory for our second demultiplexing run and sub directories for each of the fwd barcodes. 


In [ ]:
!mkdir /content/drive/MyDrive/16SColab/flongle-demult2/
!for i in /content/drive/MyDrive/16SColab/flongle-demult1/BC*; do mkdir /content/drive/MyDrive/16SColab/flongle-demult2/$(basename $i .fastq) ;done
!for i in /content/drive/MyDrive/16SColab/flongle-demult1/BC*; do mv $i /content/drive/MyDrive/16SColab/flongle-demult2/$(basename $i .fastq) ;done

Now we are ready to run the reverse demultiplex. This time we loop through each of the first demultiplex output which has seperated the reads into 5 groups based on barcodes

In [ ]:
!for i in /content/drive/MyDrive/16SColab/flongle-demult2/BC*; do porechop -i $i -b /content/drive/MyDrive/16SColab/flongle-demult2/$(basename $i .fastq);done

**Assign demultiplexed reads to the location they are from**


Below is the breakdown of which barcodes can be used to identify reads from a given location
BC01  & [BC02 + BC05] = Location 404 
[BC02 + BC06-8] & [BC03 + BC05-6]= Location 405 
[BC03 + BC07-8] & [BC04]= Loaction 923 

To join all reads from the same location into a single file we use 

```
cat indv_file_name >> combined_file_name 
```



In [ ]:
#mkdir /content/drive/MyDrive/16SColab/demultiplexed-indiv/
!mkdir /content/drive/MyDrive/16SColab/fasta-indiv
!mkdir /content/drive/MyDrive/16SColab/sam-indiv
!mkdir /content/drive/MyDrive/16SColab/samsorted-indiv
!mkdir /content/drive/MyDrive/16SColab/samoutput-indiv
!mkdir /content/drive/MyDrive/16SColab/abundanceTables-indiv

In [ ]:
!cat /content/drive/MyDrive/16SColab/flongle-demult2/BC01/BC05.fastq >> /content/drive/MyDrive/16SColab/demultiplexed-indiv/404-1.fastq
!cat /content/drive/MyDrive/16SColab/flongle-demult2/BC01/BC06.fastq >> /content/drive/MyDrive/16SColab/demultiplexed-indiv/404-2.fastq
!cat /content/drive/MyDrive/16SColab/flongle-demult2/BC01/BC07.fastq >> /content/drive/MyDrive/16SColab/demultiplexed-indiv/404-3.fastq
!cat /content/drive/MyDrive/16SColab/flongle-demult2/BC01/BC08.fastq >> /content/drive/MyDrive/16SColab/demultiplexed-indiv/404-4.fastq
!cat /content/drive/MyDrive/16SColab/flongle-demult2/BC02/BC05.fastq >> /content/drive/MyDrive/16SColab/demultiplexed-indiv/404-5.fastq

!cat /content/drive/MyDrive/16SColab/flongle-demult2/BC02/BC06.fastq >> /content/drive/MyDrive/16SColab/demultiplexed-indiv/405-1.fastq
!cat /content/drive/MyDrive/16SColab/flongle-demult2/BC02/BC07.fastq >> /content/drive/MyDrive/16SColab/demultiplexed-indiv/405-2.fastq
!cat /content/drive/MyDrive/16SColab/flongle-demult2/BC02/BC08.fastq >> /content/drive/MyDrive/16SColab/demultiplexed-indiv/405-3.fastq
!cat /content/drive/MyDrive/16SColab/flongle-demult2/BC03/BC05.fastq >> /content/drive/MyDrive/16SColab/demultiplexed-indiv/405-4.fastq
!cat /content/drive/MyDrive/16SColab/flongle-demult2/BC03/BC06.fastq >> /content/drive/MyDrive/16SColab/demultiplexed-indiv/405-5.fastq

!cat /content/drive/MyDrive/16SColab/flongle-demult2/BC03/BC07.fastq >> /content/drive/MyDrive/16SColab/demultiplexed-indiv/923-1.fastq
!cat /content/drive/MyDrive/16SColab/flongle-demult2/BC03/BC08.fastq >> /content/drive/MyDrive/16SColab/demultiplexed-indiv/923-2.fastq
!cat /content/drive/MyDrive/16SColab/flongle-demult2/BC04/BC05.fastq >> /content/drive/MyDrive/16SColab/demultiplexed-indiv/923-3.fastq
!cat /content/drive/MyDrive/16SColab/flongle-demult2/BC04/BC06.fastq >> /content/drive/MyDrive/16SColab/demultiplexed-indiv/923-4.fastq
!cat /content/drive/MyDrive/16SColab/flongle-demult2/BC04/BC07.fastq >> /content/drive/MyDrive/16SColab/demultiplexed-indiv/923-5.fastq



 # Mapping to reference database 
 
 We map to the [SILVA](https://www.arb-silva.de/) database to identify the genus of bacteria from which read is derived.

To download the required database files from SILVA from [here](https://www.arb-silva.de/no_cache/download/archive/release_138_1/Exports/)

The first step is to create a minimizer index (.mmi) of the reference sequence database. This enables faster and lower memory usage upon alignment. 

In [ ]:
!minimap2 -d /content/drive/MyDrive/16SColab/SILVA/silva.mmi /content/drive/MyDrive/16SColab/SILVA/SILVA_138.1_SSURef_NR99_tax_silva.fasta

Next we create a folder for fasta versions of our reads which will not have the quality score information. Then using sed we can create the fasta from the fastq and place them in the new directory. We use a loop to send f which is a file ending in .fastq through the pipeline. This allows us to run the line of code once rather than three times. 

```
 for f in *.fastq ; do 
```




In [ ]:
!for f in /content/drive/MyDrive/16SColab/demultiplexed-indiv/*fastq ; do sed -n '1~4s/^@/>/p;2~4p' $f > /content/drive/MyDrive/16SColab/fasta-indiv/$(basename $f .fastq).fasta ; done

Once we have the fasta files we can use the tool minimap to align them to our reference database. Using `-ax` option we tell the aligner these are ONT reads this allows it to take into account the error profile of ONT reads. The `-K` option allows us to constrain memory usage to prevent the process being killed due to requiring too much resources.



In [ ]:
!for f in /content/drive/MyDrive/16SColab/fasta-indiv/*; do  minimap2 -K 5M -ax map-ont -L /content/drive/MyDrive/16SColab/SILVA/silva.mmi $f > /content/drive/MyDrive/16SColab/sam-indiv/$(basename $f .fasta).sam ;done

# Convert SAM files to abundance tables 
The next step is to convert SAM files to format useable for converting creating abundance tables. At the start of SAM files are header lines which start with 

```
@
```
and contain information about the sequences and program used to generate SAM files.

After the header lines the lines represent information about the alignment of a query sequence to the reference (ie your mmi database). Each column has a specific meaning.

We can use the program samtools to reorder our sequences alphabetically then remove the headers as we do not need these for our purpose. 

In [ ]:
!for file in /content/drive/MyDrive/16SColab/sam-indiv/*.sam; do samtools sort -@ 2 -O sam -o /content/drive/MyDrive/16SColab/samsorted-indiv/$(basename $file).sorted $file 2> /content/drive/MyDrive/16SColab/samlog/$(basename $file).indiv.log ; done
!for file in /content/drive/MyDrive/16SColab/samsorted-indiv/*.sorted; do echo "==> ${file} <=="; grep -v '^@' "${file}" > /content/drive/MyDrive/16SColab/samoutput-indiv/$(basename $file).output; done

# Creating Abundance Tables 

Now we have aligned our reads to the reference database we need to assign each read to a taxa based on the best alignment. At the same time we need to keep a count of how many reads are assigned to a given taxa at each site. 

We can do this process repeatedly investigating different taxonomic levels. 

To do this we will use a python script that is below. 

As it is good pracitce to try to read and understand the steps in the script, I would suggest that once you have your abundance tables which should save to your Gdrive in 16S Colab -> abundanceTables folder you create a copy with a different name to save them for next week . Then you can play around with the python scripts below to help your understanding. E.G. you cangoogle commands you are unfamiliar with,edit names of variables,  add print statements to objects that you are unsure what they contain

```
print(object_name)
```

. Anything you print will be output to the box below the script. If you get errors googling these can help you solve them.  If you make any changes that you cant fix simply use Edit -> Undo .. to get back to the original version. 


Generate table of reads per genus

In [ ]:
import pandas as pd
import io
import os
import requests
import numpy as np
# load all files from the SILVA database
minit = pd.read_csv('/content/drive/MyDrive/16SColab/SILVA/seqid2taxid.map', sep='\t', index_col=0, header=None)
sild = pd.read_csv('/content/drive/MyDrive/16SColab/SILVA/tax_slv_ssu_138.1.txt', sep='\t', header=None)
sild.columns = ['tree','taxid','level','3','4']
sild['ranks'] = [x.split(';')[-2] for x in sild.tree.values]
sild['tree'] = [x[0:-1] for x in sild.tree]
sild.index = sild.taxid
namesdmp = pd.read_csv('/content/drive/MyDrive/16SColab/SILVA/names.dmp', sep='\t', index_col=0, header=None)
# choose rank and month
ranks = 'genus'
# choose dir of sam files
dirc = '/content/drive/MyDrive/16SColab/samoutput-indiv/' 
# create 
nr = 0
for filename in os.listdir(dirc):
    print(filename)
    
    try:
        silva_10k = pd.read_csv('/content/drive/MyDrive/16SColab/samoutput-indiv/%s' %filename, 
                         sep='\t', header=None, usecols = [0,2,4,13])
    except: 
        continue
    
    silva_10k.columns = ['Read_ID', 'id','MS', 'ASs']
    silva_10k['ASs'] = silva_10k['ASs'].astype('str')
    silva_10k['AS'] = [x.split(':i:')[-1] for x in silva_10k['ASs'].values]
    silva_10k.dropna(axis=0, subset=['AS'], inplace=True)
    silva_10k['AS'] = silva_10k['AS'].astype('float')
    mini = silva_10k[silva_10k['AS'] == silva_10k.groupby('Read_ID')['AS'].transform('max')]
    mini = mini[['Read_ID', 'MS', 'AS','id']]              
    mini.columns = ['read','score','as','id']
    mini = mini[~mini.id.isnull()]
    mini['taxid'] = minit.loc[mini.id.values].values
    mini['tree'] = sild.loc[mini.taxid.values]['tree'].values
    mini['level'] = sild.loc[mini.taxid.values]['level'].values
    mini['phylum'] = [x.split(';')[0] for x in mini.tree]
    print(mini.phylum.unique())
    # if genus
    if ranks == 'genus':
        mini = mini[mini.level=='genus'] 
        mini['ranks'] = namesdmp.loc[mini.taxid][2].values 
        mini.index = mini.read  
        for i in mini.index[mini.duplicated(subset='read', keep=False)].unique():
            minil = list(mini.loc[i].taxid.values)
            if minil.count(minil[0]) != len(minil):
                mini.drop(i)
        mini.drop_duplicates(subset='read', keep='first', inplace=True)
    # if family
    if ranks == 'family':
        mini = mini[(mini.level=='genus')|(mini.level=='family')] # 9349
        mini['ranks'] = namesdmp.loc[mini.taxid][2].values   
        mini.index = mini.read
        for i in mini.index[mini.duplicated(subset='read', keep=False)].unique():
            minil = list(mini.loc[i].taxid.values)
            if minil.count(minil[0]) != len(minil):
                minil2 = [x.split(';')[-2] for x in mini.loc[i].tree.values]
                if minil2.count(minil2[0]) == len(minil2):
                    #print('3')
                    mini['ranks'].loc[i] = minil2[0]
                else: 
                    mini.drop(i)
        mini.drop_duplicates(subset='read', keep='first', inplace=True)           
        mini['genuss'] = [sild.loc[x].tree.split(';')[-2] for x in mini.taxid]
        mini['ranks'][mini.level=='genus'] = mini['genuss'][mini.level=='genus']
    mini2 = pd.DataFrame(mini.ranks.value_counts())
    if nr==0:
        #print('4')
        minif = mini2.copy(deep=True)
        minif.columns.values[nr] = filename.split('.')[0]
    else:
        minif = minif.merge(mini2, left_index=True, right_index=True, how='outer')
        minif.columns.values[nr] = filename.split('.')[0]
    nr = nr+1
# describe all missing bacteria as absent
minif = minif.fillna(0) 
#Above causes warning cant figure out best way to unchain
# save the final txt files  
minif.to_csv('/content/drive/MyDrive/16SColab/abundanceTables-indiv/minimap2_k12_%s.txt' %ranks, sep='\t')

Generate reads per family table 

In [ ]:
import pandas as pd
import io
import os
import requests
import numpy as np
# load all files from the SILVA database
minit = pd.read_csv('/content/drive/MyDrive/16SColab/SILVA/seqid2taxid.map', sep='\t', index_col=0, header=None)
sild = pd.read_csv('/content/drive/MyDrive/16SColab/SILVA/tax_slv_ssu_138.1.txt', sep='\t', header=None)
sild.columns = ['tree','taxid','level','3','4']
sild['ranks'] = [x.split(';')[-2] for x in sild.tree.values]
sild['tree'] = [x[0:-1] for x in sild.tree]
sild.index = sild.taxid
namesdmp = pd.read_csv('/content/drive/MyDrive/16SColab/SILVA/names.dmp', sep='\t', index_col=0, header=None)
# choose rank and month
ranks = 'family'
# choose dir of sam files
dirc = '/content/drive/MyDrive/16SColab/samoutput-indiv/' 
# create 
nr = 0
for filename in os.listdir(dirc):
    print(filename)
    
    try:
        silva_10k = pd.read_csv('/content/drive/MyDrive/16SColab/samoutput-indiv/%s' %filename, 
                         sep='\t', header=None, usecols = [0,2,4,13])
    except: 
        continue
    
    silva_10k.columns = ['Read_ID', 'id','MS', 'ASs']
    silva_10k['ASs'] = silva_10k['ASs'].astype('str')
    silva_10k['AS'] = [x.split(':i:')[-1] for x in silva_10k['ASs'].values]
    silva_10k.dropna(axis=0, subset=['AS'], inplace=True)
    silva_10k['AS'] = silva_10k['AS'].astype('float')
    mini = silva_10k[silva_10k['AS'] == silva_10k.groupby('Read_ID')['AS'].transform('max')]
    mini = mini[['Read_ID', 'MS', 'AS','id']]              
    mini.columns = ['read','score','as','id']
    mini = mini[~mini.id.isnull()]
    mini['taxid'] = minit.loc[mini.id.values].values
    mini['tree'] = sild.loc[mini.taxid.values]['tree'].values
    mini['level'] = sild.loc[mini.taxid.values]['level'].values
    mini['phylum'] = [x.split(';')[0] for x in mini.tree]
    print(mini.phylum.unique())
    # if genus
    if ranks == 'genus':
        mini = mini[mini.level=='genus'] 
        mini['ranks'] = namesdmp.loc[mini.taxid][2].values 
        mini.index = mini.read  
        for i in mini.index[mini.duplicated(subset='read', keep=False)].unique():
            minil = list(mini.loc[i].taxid.values)
            if minil.count(minil[0]) != len(minil):
                mini.drop(i)
        mini.drop_duplicates(subset='read', keep='first', inplace=True)
    # if family
    if ranks == 'family':
        mini = mini[(mini.level=='genus')|(mini.level=='family')] # 9349
        mini['ranks'] = namesdmp.loc[mini.taxid][2].values   
        mini.index = mini.read
        for i in mini.index[mini.duplicated(subset='read', keep=False)].unique():
            minil = list(mini.loc[i].taxid.values)
            if minil.count(minil[0]) != len(minil):
                minil2 = [x.split(';')[-2] for x in mini.loc[i].tree.values]
                if minil2.count(minil2[0]) == len(minil2):
                    mini['ranks'].loc[i] = minil2[0]
                else: 
                    mini.drop(i)
        mini.drop_duplicates(subset='read', keep='first', inplace=True)           
        mini['genuss'] = [sild.loc[x].tree.split(';')[-2] for x in mini.taxid]
        mini['ranks'][mini.level=='genus'] = mini['genuss'][mini.level=='genus']
    mini2 = pd.DataFrame(mini.ranks.value_counts())
    if nr==0:
        minif = mini2.copy(deep=True)
        minif.columns.values[nr] = filename.split('.')[0]
    else:
        minif = minif.merge(mini2, left_index=True, right_index=True, how='outer')
        minif.columns.values[nr] = filename.split('.')[0]
    nr = nr+1
    
# describe all missing bacteria as absent
minif = minif.fillna(0) 
#Above causes warning cant figure out best way to unchain
# save the final txt files  
minif.to_csv('/content/drive/MyDrive/16SColab/abundanceTables-indiv/minimap2_k12_%s.txt' %ranks, sep='\t')

Generate reads per order 

In [ ]:
import pandas as pd
import io
import os
import requests
import numpy as np
# load all files from the SILVA database
minit = pd.read_csv('/content/drive/MyDrive/16SColab/SILVA/seqid2taxid.map', sep='\t', index_col=0, header=None)
sild = pd.read_csv('/content/drive/MyDrive/16SColab/SILVA/tax_slv_ssu_138.1.txt', sep='\t', header=None)
sild.columns = ['tree','taxid','level','3','4']
sild['ranks'] = [x.split(';')[-2] for x in sild.tree.values]
sild['tree'] = [x[0:-1] for x in sild.tree]
sild.index = sild.taxid
namesdmp = pd.read_csv('/content/drive/MyDrive/16SColab/SILVA/names.dmp', sep='\t', index_col=0, header=None)
# choose rank and month
ranks = 'family'
# choose dir of sam files
dirc = '/content/drive/MyDrive/16SColab/samoutput-indiv/' 
# create 
nr = 0
for filename in os.listdir(dirc):
    print(filename)
    
    try:
        silva_10k = pd.read_csv('/content/drive/MyDrive/16SColab/samoutput-indiv/%s' %filename, sep='\t', header=None, usecols = [0,2,4,13])
    except: 
        continue
    
    silva_10k.columns = ['Read_ID', 'id','MS', 'ASs']
    silva_10k['ASs'] = silva_10k['ASs'].astype('str')
    silva_10k['AS'] = [x.split(':i:')[-1] for x in silva_10k['ASs'].values]
    silva_10k.dropna(axis=0, subset=['AS'], inplace=True)
    silva_10k['AS'] = silva_10k['AS'].astype('float')
    mini = silva_10k[silva_10k['AS'] == silva_10k.groupby('Read_ID')['AS'].transform('max')]
    mini = mini[['Read_ID', 'MS', 'AS','id']]              
    mini.columns = ['read','score','as','id']
    mini = mini[~mini.id.isnull()]
    mini['taxid'] = minit.loc[mini.id.values].values
    mini['tree'] = sild.loc[mini.taxid.values]['tree'].values
    mini['level'] = sild.loc[mini.taxid.values]['level'].values
    mini['phylum'] = [x.split(';')[0] for x in mini.tree]
    print(mini.phylum.unique())
    # if genus
    if ranks == 'genus':
        mini = mini[mini.level=='genus'] 
        mini['ranks'] = namesdmp.loc[mini.taxid][2].values 
        mini.index = mini.read  
        for i in mini.index[mini.duplicated(subset='read', keep=False)].unique():
            minil = list(mini.loc[i].taxid.values)
            if minil.count(minil[0]) != len(minil):
                mini.drop(i)
        mini.drop_duplicates(subset='read', keep='first', inplace=True)
    # if family
    if ranks == 'family':
        mini = mini[(mini.level=='genus')|(mini.level=='family')] # 9349
        mini['ranks'] = namesdmp.loc[mini.taxid][2].values   
        mini.index = mini.read
        for i in mini.index[mini.duplicated(subset='read', keep=False)].unique():
            minil = list(mini.loc[i].taxid.values)
            minil2 = [x.split(';')[-3] for x in mini.loc[i].tree.values]
            mini['ranks'].loc[i] = minil2[0]
        mini.drop_duplicates(subset='read', keep='first', inplace=True)           
        mini['familys'] = [sild.loc[x].tree.split(';')[-3] for x in mini.taxid]
        mini['ranks'][mini.level=='family'] = mini['familys'][mini.level=='family']
    mini2 = pd.DataFrame(mini.familys.value_counts())
    if nr==0:
        minif = mini2.copy(deep=True)
        minif.columns.values[nr] = filename.split('.')[0]
    else:
        minif = minif.merge(mini2, left_index=True, right_index=True, how='outer')
        minif.columns.values[nr] = filename.split('.')[0]
        minif = minif.copy(deep=True)
    nr = nr+1
    
# describe all missing bacteria as absent
minif = minif.fillna(0) 
#Above causes warning cant figure out best way to unchain. 
# save the final txt files  
minif.to_csv('/content/drive/MyDrive/16SColab/abundanceTables-indiv/minimap2_k12_order.txt', sep='\t')

A python script to output taxanomic ID as well as reads assigned per taxon. This is useful for later steps in annotation of figures. If you could like to challenge your understanding of the code you can try to adapt the script to output the taxanomic id for higher taxa levels (currently set to Genus). 

In [ ]:
import pandas as pd
import io
import os
import requests
import numpy as np
# load all files from the SILVA database
minit = pd.read_csv('/content/drive/MyDrive/16SColab/SILVA/seqid2taxid.map', sep='\t', index_col=0, header=None)
sild = pd.read_csv('/content/drive/MyDrive/16SColab/SILVA/tax_slv_ssu_138.1.txt', sep='\t', header=None)
sild.columns = ['tree','taxid','level','3','4']
sild['ranks'] = [x.split(';')[-2] for x in sild.tree.values]
sild['tree'] = [x[0:-1] for x in sild.tree]
sild.index = sild.taxid
namesdmp = pd.read_csv('/content/drive/MyDrive/16SColab/SILVA/names.dmp', sep='\t', index_col=0, header=None)
# choose rank and month
ranks = 'genus'
# choose dir of sam files
dirc = '/content/drive/MyDrive/16SColab/samoutput-indiv/' 
# create 
nr = 0
for filename in os.listdir(dirc):
    print(filename)
    
    try:
        silva_10k = pd.read_csv('/content/drive/MyDrive/16SColab/samoutput-indiv/%s' %filename, 
                         sep='\t', header=None, usecols = [0,2,4,13])
    except: 
        continue
    
    silva_10k.columns = ['Read_ID', 'id','MS', 'ASs']
    silva_10k['ASs'] = silva_10k['ASs'].astype('str')
    silva_10k['AS'] = [x.split(':i:')[-1] for x in silva_10k['ASs'].values]
    silva_10k.dropna(axis=0, subset=['AS'], inplace=True)
    silva_10k['AS'] = silva_10k['AS'].astype('float')
    mini = silva_10k[silva_10k['AS'] == silva_10k.groupby('Read_ID')['AS'].transform('max')]
    mini = mini[['Read_ID', 'MS', 'AS','id']]              
    mini.columns = ['read','score','as','id']
    mini = mini[~mini.id.isnull()]
    mini['taxid'] = minit.loc[mini.id.values].values
    mini['tree'] = sild.loc[mini.taxid.values]['tree'].values
    mini['level'] = sild.loc[mini.taxid.values]['level'].values
    mini['phylum'] = [x.split(';')[0] for x in mini.tree]
    #print(mini.phylum.unique())
    # if genus
    if ranks == 'genus':
        mini = mini[mini.level=='genus'] 
        mini['ranks'] = namesdmp.loc[mini.taxid][2].values 
        mini.index = mini.read  
        for i in mini.index[mini.duplicated(subset='read', keep=False)].unique():
            minil = list(mini.loc[i].taxid.values)
            if minil.count(minil[0]) != len(minil):
                mini.drop(i)
        mini.drop_duplicates(subset='read', keep='first', inplace=True)
    # if family
    #print(mini)
    rank_taxid = mini[["ranks", "taxid"]]
    mini2 = pd.DataFrame(mini.ranks.value_counts())

    if nr==0:
        #print('4')
        minif = mini2.copy(deep=True)
        minif.columns.values[nr] = filename.split('.')[0]
        mini3 = mini.ranks.value_counts().rename_axis('ranks').to_frame(filename.split('.')[0])
        mini4 = pd.merge(mini3, rank_taxid, on="ranks", how="left")
        mini5 = mini4.drop_duplicates()
        minix= mini5.copy(deep=True)
    else:
        minif = minif.merge(mini2, left_index=True, right_index=True, how='outer')
        minif.columns.values[nr] = filename.split('.')[0]
        mini3 = mini.ranks.value_counts().rename_axis('ranks').to_frame(filename.split('.')[0])

        merged = pd.merge(minix, mini3, how='inner', left_on=['ranks'], right_on=['ranks'])
        minix= merged
        """
        miniz = pd.merge(minix, mini3, on="ranks")
        print(miniz)
        minix.columns.values[nr] = filename.split('.')[0]
        miniq = miniz.loc[:, ['ranks', 'taxid', 'mock3','mock5',]]
        miniq2=miniq.rename(columns = {'reads_y':'mock3','reads_x':'mock5'})

        print(miniq)
        """
        
    nr = nr+1
    
# describe all missing bacteria as absent
minif = minif.fillna(0) 
minix = minix.fillna(0) 
#Above causes warning cant figure out best way to unchain
# save the final txt files  
df = pd.DataFrame(merged)
temp_cols=df.columns.tolist()
index=df.columns.get_loc("taxid")
new_cols=temp_cols[index:index+1] + temp_cols[0:index] + temp_cols[index+1:]
df=df[new_cols]
minif.to_csv('minimap2_k12_%s.txt' %ranks, sep='\t')
df.to_csv('/content/drive/MyDrive/16SColab/abundanceTables-indiv/minimap2_k12_%s_taxids.txt' %ranks, sep='\t')


# End of python based session

Now we have our abundance tables we are going to look into comparisons of diversity and abundance of taxa across sites. 


For more examples of bioinformatic pipelines in colab check out this tutorial from another [lab](https://colab.research.google.com/drive/1BK2-d7xcF1puNO4h1lFgRMhgnKAuTKzh?usp=sharing).